In [1]:
import nuclio 
import mlrun

In [2]:
import storey

In [3]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

In [4]:
%%nuclio cmd -c
pip install beautifulsoup4
pip install pandas
pip install v3io_frames
pip install storey==0.4.0

In [5]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [6]:
%nuclio cmd -c pip install v3io

In [7]:
# nuclio: start-code

In [7]:
import mlrun.feature_store as fs
from mlrun.feature_store.steps import *
import mlrun
import pandas as pd
import numpy
import os
import math

In [8]:
def rm_inf(data):
    print(data)
    data = [0 if item == math.inf or item == -math.inf else item for item in data ]
    #data = [0 for item in data if item == -math.inf]
    return data

In [119]:
def handler(context, event):
    context.logger.info("Reading features from feature vector")
    
    # Reading the data from feature service
    df = fs.get_offline_features("stocks-vec").to_dataframe()
    df = df.dropna()
    
    data = df.values
    
    columns = [{'text': key, 'type': 'object'} for key in df.columns]
    
    response = [{'columns': columns,
                'rows': data,
                'type': 'table'}]
    return response

In [120]:
def init_context(context):
    # Setting our enviornment
    context.logger.info("Initalizing context & constructing featrure vector")
    setattr(context, 'PROJECT_NAME', os.getenv('PROJECT_NAME', 'stocks-' + os.getenv('V3IO_USERNAME')))
    mlrun.set_environment(project=context.PROJECT_NAME)
    
    sym_to_url = {'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc',
                  'AAPL': 'apple-computer-inc', 'INTC' : 'intel-corp'}
    setattr(context, 'sym_to_url', sym_to_url)
    setattr(context, 'stocks_vec', "stocks-vec")
    
    # Declaring the features we would like to add to our vector
    features = ["stocks.Closes_min_1h",
                "stocks.Closes_max_1h",
                "stocks.Volumes_min_1h",
                "stocks.Volumes_max_1h",
                "stocks.Open",
                "stocks.High",
                "stocks.Low",
                "stocks.Close",
                "stocks.Volume",
#                 "news.Content",
#                 "news.Link",
                "news.Sentiment"]
#     features = ["stocks.*", "news.*"]
    
    # Making our vector
    vector = fs.FeatureVector(context.stocks_vec, features, description="stocks demo feature vector")
    vector.save()

In [121]:
# nuclio: end-code

In [117]:
# test local
init_context(context)
event = ""#nuclio.Event(body='')
df = handler(context,event)

Python> 2021-06-23 09:26:06,339 [info] Initalizing context & constructing featrure vector
Python> 2021-06-23 09:26:06,426 [info] Reading features from feature vector


In [124]:
# test remote deployment
from mlrun import code_to_function
fn = code_to_function('vector_reader',
                      handler='handler')

# Set parameters for current deployment
fn.set_envs({'PROJECT_NAME' : 'stocks-dani',
             'STOCKS_STREAM':  os.getenv('V3IO_USERNAME') +' /stocks/stocks_stream'})
fn.spec.max_replicas = 2

In [125]:
addr = fn.deploy(project='stocks-dani')

> 2021-06-23 09:27:46,711 [info] Starting remote function deploy
2021-06-23 09:27:47  (info) Deploying function
2021-06-23 09:27:47  (info) Building
2021-06-23 09:27:48  (info) Staging files and preparing base images
2021-06-23 09:27:48  (info) Building processor image
2021-06-23 09:28:08  (info) Build complete
2021-06-23 09:28:21  (info) Function deploy complete
> 2021-06-23 09:28:21,884 [info] function deployed, address=default-tenant.app.vmdev31.lab.iguazeng.com:32572


In [126]:
!curl {addr}

[{"columns": [{"text": "Closes_min_1h", "type": "object"}, {"text": "Closes_max_1h", "type": "object"}, {"text": "Volumes_min_1h", "type": "object"}, {"text": "Volumes_max_1h", "type": "object"}, {"text": "Open", "type": "object"}, {"text": "High", "type": "object"}, {"text": "Low", "type": "object"}, {"text": "Close", "type": "object"}, {"text": "Volume", "type": "object"}, {"text": "Sentiment", "type": "object"}], "rows": "array([[2.61696014e+02, 2.62312805e+02, 1.31510000e+04, ...,\n        2.62312805e+02, 2.14820000e+04, 0.00000000e+00],\n       [2.61696014e+02, 2.62359985e+02, 1.31510000e+04, ...,\n        2.62359985e+02, 4.39850000e+04, 0.00000000e+00],\n       [2.61696014e+02, 2.62359985e+02, 1.31510000e+04, ...,\n        2.62309998e+02, 2.34850000e+04, 0.00000000e+00],\n       ...,\n       [2.65079987e+02, 2.65769989e+02, 2.50130000e+04, ...,\n        2.65130005e+02, 1.18529000e+05, 0.00000000e+00],\n       [2.43975000e+03, 2.44800000e+03, 7.21000000e+02, ...,\n        2.446610